<a href="https://colab.research.google.com/github/yonasyifter/Account-Storage/blob/master/DL_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zeeshier/weather-forecast-dataset")

print("Path to dataset files:", path)

100%|██████████| 114k/114k [00:00<00:00, 376kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/zeeshier/weather-forecast-dataset/versions/1


In [4]:
import pandas as pd

In [5]:
import os

# List all files in the downloaded directory
files_in_dataset = os.listdir(path)

# Filter for CSV files
csv_files = [f for f in files_in_dataset if f.endswith('.csv')]

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in the dataset directory: {path}")

# Assuming the first CSV file found is the correct one for simplicity.
# If there are multiple CSVs and specific one is needed, further logic may be required.
first_csv_file = csv_files[0]
full_csv_path = os.path.join(path, first_csv_file)

df = pd.read_csv(full_csv_path)

print(f"Successfully loaded '{first_csv_file}'.")

Successfully loaded 'weather_forecast_data.csv'.


In [6]:
df.head()

,Temperature,Humidity,Wind_Speed,Cloud_Cover,Pressure,Rain
0,23.720338,89.592641,7.335604,50.501694,1032.378759,rain
1,27.879734,46.489704,5.952484,4.990053,992.614190,no rain
2,25.069084,83.072843,1.371992,14.855784,1007.231620,no rain
3,23.622080,74.367758,7.050551,67.255282,982.632013,rain
4,20.591370,96.858822,4.643921,47.676444,980.825142,no rain


In [7]:
df.isnull().sum()

,0
Temperature,0
Humidity,0
Wind_Speed,0
Cloud_Cover,0
Pressure,0
Rain,0


In [8]:
df = df.dropna()

In [9]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [12]:
X = df[[
    "Temperature",
    "Humidity",
    "Wind_Speed",
    "Cloud_Cover",
    "Pressure"
]]

In [13]:
Y = df["Rain"]

In [14]:
# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [15]:
X_scaled

array([[ 0.1554307 ,  1.26539348, -0.44481385,  0.02897213,  0.8947143 ],
       [ 0.72322527, -0.89507374, -0.6841429 , -1.53407371, -1.07457032],
       [ 0.33954662,  0.93859882, -1.47673073, -1.19524633, -0.35066253],
       ...,
       [ 0.75442305, -1.0290299 , -1.21282806,  0.89928899, -0.75241945],
       [-1.0637386 , -0.32274425, -1.3032236 , -1.6237562 ,  1.59414185],
       [ 0.54229936,  1.64176416,  1.49793905,  1.08868017, -0.62316338]])

In [16]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, Y, test_size=0.2, shuffle=False
)

# Small NN (TFLite optimized)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(1)
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Model Using Small Nueral Network

In [18]:
y_train = y_train.map({'no rain': 0, 'rain': 1})
y_test = y_test.map({'no rain': 0, 'rain': 1})

# Small NN (TFLite optimized)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(1)
])

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=8,
    validation_data=(X_test, y_test),
    verbose=1
)

Epoch 1/30


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1195 - mae: 0.2534 - val_loss: 0.0544 - val_mae: 0.1640
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0500 - mae: 0.1551 - val_loss: 0.0478 - val_mae: 0.1451
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0474 - mae: 0.1436 - val_loss: 0.0463 - val_mae: 0.1407
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0455 - mae: 0.1397 - val_loss: 0.0442 - val_mae: 0.1363
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0398 - mae: 0.1288 - val_loss: 0.0420 - val_mae: 0.1282
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0416 - mae: 0.1276 - val_loss: 0.0401 - val_mae: 0.1253
Epoch 7/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0375 - mae: 0.1214 - val_loss: 0.0388 - val_mae: 0.1225
Epoch 8/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0345 - mae: 0.1142 - val_loss: 0.0373 - val_mae: 0.1177
Epoch 9/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0376 

In [19]:
new_data = pd.DataFrame({
    'Temperature': [29],
    'Humidity': [65],
    'Wind_Speed': [5.0],  # Assuming a value, as it's not provided
    'Cloud_Cover': [30.0], # Assuming a value
    'Pressure': [1010.0]  # Assuming a value
})

# Scale the new data using the *same* scaler fitted on the training data
new_data_scaled = scaler.transform(new_data)

# Make a prediction
prediction_raw = model.predict(new_data_scaled)

# Interpret the prediction
# Assuming a threshold of 0.5 (values >= 0.5 predict 'rain', < 0.5 predict 'no rain')
prediction_label = 'rain' if prediction_raw[0][0] >= 0.5 else 'no rain'

print(f"Raw prediction output: {prediction_raw[0][0]:.4f}")
print(f"Based on the input (Temperature: {new_data['Temperature'].values[0]}, Humidity: {new_data['Humidity'].values[0]}, etc.), the model predicts: {prediction_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Raw prediction output: -0.0581
Based on the input (Temperature: 29, Humidity: 65, etc.), the model predicts: no rain


# Model Using Random-Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the RandomForestClassifier
# You can tune hyperparameters like n_estimators (number of trees) and random_state
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set and evaluate accuracy
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"RandomForest Model Accuracy on Test Set: {accuracy_rf:.4f}")

RandomForest Model Accuracy on Test Set: 1.0000


In [21]:
# Make a prediction with the RandomForest model using the scaled new data
prediction_rf_raw = rf_model.predict(new_data_scaled)

# Interpret the prediction (0 for no rain, 1 for rain)
prediction_rf_label = 'rain' if prediction_rf_raw[0] == 1 else 'no rain'

print(f"Raw RandomForest prediction output (numerical): {prediction_rf_raw[0]}")
print(f"Based on the input, the RandomForest model predicts: {prediction_rf_label}")

Raw RandomForest prediction output (numerical): 0
Based on the input, the RandomForest model predicts: no rain


In [23]:
from sklearn.metrics import accuracy_score

# Make predictions with the Neural Network model on the test set
y_pred_nn_raw = model.predict(X_test)

# Convert raw predictions to binary (0 or 1) using a 0.5 threshold
y_pred_nn = (y_pred_nn_raw >= 0.5).astype(int)

# Calculate accuracy for the Neural Network
accuracy_nn = accuracy_score(y_test, y_pred_nn)

print(f"Neural Network Model Accuracy on Test Set: {accuracy_nn:.4f}")
print(f"RandomForest Model Accuracy on Test Set: {accuracy_rf:.4f}")

print("\nComparison:")
if accuracy_nn > accuracy_rf:
    print(f"The Neural Network model has a higher accuracy of {accuracy_nn:.4f} compared to RandomForest's {accuracy_rf:.4f}.")
elif accuracy_rf > accuracy_nn:
    print(f"The RandomForest model has a higher accuracy of {accuracy_rf:.4f} compared to Neural Network's {accuracy_nn:.4f}.")
else:
    print(f"Both models have the same accuracy of {accuracy_nn:.4f}.")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Neural Network Model Accuracy on Test Set: 0.9800
RandomForest Model Accuracy on Test Set: 1.0000

Comparison:
The RandomForest model has a higher accuracy of 1.0000 compared to Neural Network's 0.9800.
